In [ ]:
import pandas as pd
import numpy as np
import json
import random
import nltk
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import text_to_word_sequence
from tensorflow import keras
from keras import preprocessing

# Uploading and preprocessing data

In [ ]:
#lowercase and removing stopwords
def get_data(filename):
    with open(filename, 'r', encoding='unicode_escape') as f:
        data = f.read()
    data = data.lower()

    nltk.download('stopwords')
    stop_words = stopwords.words('english')
    data = data.split()
    data = [w for w in data if w not in stop_words]
    data = " ".join(data)

    return(data)

In [ ]:
it_data = get_data('it_oriented.TXT')
non_it_data = get_data('ordinary.TXT')
all_data = it_data + non_it_data

all_data[:20]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'reason java still re'

In [ ]:
#creating word index
def create_index(texts, filename):
    words = texts.split() 
    
    #there can be up to 10 000 words understood by our neural network
    tokenizer = Tokenizer(num_words=10000) 
    
    #we are tokenizing all of those words from the text files
    tokenizer.fit_on_texts(words) 
    sequences = tokenizer.texts_to_sequences(words)
    word_index = tokenizer.word_index

    #print(f"Found {len(word_index)} unique words.") 
    
    with open (filename, 'w') as f:
        json.dump(word_index, f, indent=4)

In [ ]:
create_index(all_data, 'word_index.json')

In [ ]:
#grab and return word index
def get_index(filename): #for opening that json file
    with open(filename, 'r') as f:
        data = json.load(f)
    return(data)

In [ ]:
word_index = get_index('word_index.json')

In [ ]:
#creating sentences
def create_sents(text):
    nltk.download('punkt')
    sentences = nltk.tokenize.sent_tokenize(text)
    return(sentences)

In [ ]:
it_sents = create_sents(it_data)
non_it_sents = create_sents(non_it_data)

it_sents[0]

In [ ]:
#padding
def padding_data(sentences, index, maxlen=25):
    new_sentences = []
    for sentence in sentences:
        #this will give us a sentence converted to numerical array
        sentence = text_to_word_sequence(sentence) 
        new_sentence = []
        words = []
        for word in sentence:
            try:
                word = index[word]
            except:
                KeyError
                #for unknown words we encounter:
                word = 0 
            words.append(word)
        new_sentence.append(words)
        new_sentence = preprocessing.sequence.pad_sequences(new_sentence, maxlen=maxlen, padding='post')
        new_sentences.append(new_sentence[0])
    return(new_sentences)

In [ ]:
it_padded = padding_data(it_sents, word_index, maxlen=25)
non_it_padded = padding_data(non_it_sents, word_index, maxlen=25)

it_padded[0]

array([ 331,  166,   67,  998, 3896, 3897,   98, 3898,  763,  365,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0], dtype=int32)

In [ ]:
#reverse word index
def reverse_index(word_index):
    reverse_word_index = {value: key for (key, value) in word_index.items()}
    return(reverse_word_index)
    
def reconst_text(text, reverse_word_index):
    return(" ".join([reverse_word_index.get(i, "?") for i in text]))

In [ ]:
reverse_word_index = reverse_index(word_index)

reconst_text(it_padded[0], reverse_word_index)

'reason java still relevant shoved throats high schoolers college students ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?'

# Labelling Data

In [ ]:
def label_data(sentences, label):
    total_chunks = []
    for sentence in sentences:
        #in form of a tuple that consists of a list of numbers and a label
        total_chunks.append((sentence, label)) 
    return(total_chunks)

In [ ]:
non_it_labelled = label_data(non_it_padded, 0)
it_labelled = label_data(it_padded, 1)

non_it_labelled[0]

(array([2070, 1527, 5745, 5746,  304, 5747,  986,  501, 5748, 3376,  137,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0], dtype=int32), 0)

# Creating Training Data

In [ ]:
def create_training(total_chunks, cutoff):
    random.shuffle(total_chunks)
    training_data = []
    training_labels = []
    testing_data = []
    testing_labels = []
    test_num = len(total_chunks) * cutoff
    x = 0
    
    for entry in total_chunks:
        if x > test_num:
            testing_data.append(entry[0])
            testing_labels.append(entry[1])
        else:
            training_data.append(entry[0])
            training_labels.append(entry[0])
        x = x + 1

    #we are converting all that into a numpy array
    training_data = np.array(training_data) 
    training_labels = np.array(training_labels)
    testing_data = np.array(testing_data)
    testing_labels = np.array(testing_labels)
    
    return(training_data, training_labels, testing_data, testing_labels)

In [ ]:
all_data = it_labelled + non_it_labelled
tt_data = create_training(all_data, cutoff = 0.5)

#why only 4??????
print(len(all_data))
print(len(tt_data))

3566
4
